In [ ]:
from Chapter_08_MT5 import *
import ta
import numpy as np
import warnings
from datetime import datetime
import pandas as pd
import MetaTrader5 as mt5
warnings.filterwarnings("ignore")
mt5.initialize()


def svm_reg_trading(symbol):

    def feature_engineering(df):
        """ Create new variables"""

        # We copy the dataframe to avoid interferences in the data
        df_copy = df.dropna().copy()

        # Create the returns
        df_copy["returns"] = df_copy["close"].pct_change(1)

        # Create the SMAs
        df_indicators = ta.add_all_ta_features(
        df, open="open", high="high", low="low", close="close", volume="volume", fillna=True).shift(1)

        dfc = pd.concat((df_indicators, df_copy), axis=1)

        return dfc.dropna()


    # Import the data
    df = MT5.get_data(symbol, 3500)[["open", "high", "low", "close", "tick_volume"]]

    df.columns = ["open", "high", "low", "close", "volume"]

    dfc = feature_engineering(df)

    # Percentage train set
    split = int(0.99*len(dfc))

    # Train set creation
    X_train = dfc.iloc[:split,6:dfc.shape[1]-1]
    y_train = dfc[["returns"]].iloc[:split]


    # Test set creation
    X_test = dfc.iloc[split:,6:dfc.shape[1]-1]
    y_test = dfc[["returns"]].iloc[split:]


    # What you need to remind about this chapter
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()

    X_train_sc = sc.fit_transform(X_train)
    X_test_sc = sc.transform(X_test)


    from sklearn.decomposition import PCA
    pca = PCA(n_components=6)
    X_train_pca = pca.fit_transform(X_train_sc)
    X_test_pca = pca.transform(X_test_sc)

    # Import the class
    from sklearn.svm import SVR

    # Initialize the class
    reg = SVR()

    # Fit the model
    reg.fit(X_train_pca, y_train)

    # Create predictions for the whole dataset
    X = np.concatenate((X_train_pca, X_test_pca), axis=0)

    dfc["prediction"] = reg.predict(X)

    buy = dfc["prediction"].iloc[-1]>0
    sell = not buy
    return buy, sell

mt5.initialize()
# True = Live Trading and False = Screener
live = True

if live:
    current_account_info = mt5.account_info()
    print("------------------------------------------------------------------")
    print("Date: ", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
    print(f"Balance: {current_account_info.balance} USD, \t"
          f"Equity: {current_account_info.equity} USD, \t"
          f"Profit: {current_account_info.profit} USD")
    print("------------------------------------------------------------------")


info_order = {
    "RUSSEL 2000": ["EURUSD", 1.],
    "Bitcoin": ["EURUSD", 0.1],
    "Nasdaq 100": ["EURUSD", 0.3]
}


start = datetime.now().strftime("%H:%M:%S")
while True:
    # Verfication for launch
    if datetime.now().weekday() not in (1,2):
        is_time = datetime.now().strftime("%H:%M:%S") == start #"23:59:59"
    else:
        is_time = False

    
    # Launch the algorithm
    if is_time:

        # Open the trades
        for asset in info_order.keys():

            # Initialize the inputs
            symbol = info_order[asset][0]
            lot = info_order[asset][1]

            # Create the signals
            buy, sell = svm_reg_trading(symbol)

             # Run the algorithm
            if live:
                MT5.run(symbol, buy, sell,lot)

            else:
                print(f"Symbol: {symbol}\t"
                     f"Buy: {buy}\t"
                     f"Sell: {sell}")
        time.sleep(1)

------------------------------------------------------------------
Date:  2021-11-20 11:01:06
Balance: 10000.0 USD, 	Equity: 10000.0 USD, 	Profit: 0.0 USD
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2021-11-20 11:01:08
SYMBOL: EURUSD
BUY: True 	  SHORT: False
POSITION: None 	 ID: None
OPEN LONG TRADE: Market closed
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2021-11-20 11:01:09
SYMBOL: EURUSD
BUY: True 	  SHORT: False
POSITION: None 	 ID: None
OPEN LONG TRADE: Market closed
------------------------------------------------------------------
------------------------------------------------------------------
Date:  2021-11-20 11:01:10
SYMBOL: EURUSD
BUY: True 	  SHORT: False
POSITION: None 	 ID: None
OPEN LONG TRADE: Market closed
------------------------------------------------------------------


KeyboardInterrupt: 